# Проект: Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля. 
Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.

Критерии важные заказчику:

- качество предсказания;
- время обучения;
- скорость предсказания.

**Цель проекта:** разработать модель машинного обучения способную наилучшим образом предсказать рыночную стоимость автомобиля.

## Обзор данных

Импортируем библиотеки и подробнее изучим данные.

### Импортируем данные

In [46]:
# Импорт стандартных библиотек
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Импорт внешних библиотек
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [47]:
df = pd.read_csv('/datasets/autos.csv')

### Обзор данных

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [49]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [50]:
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


### Вывод:
    

Из имеющихся данных видно, что:

* датасет состоит из 354 369 строк;
* в данных имеются пропуски;
* формат даты необходимо преобразовать, если он будет использоваться;
* всего мы имеем 15 фич, 6 из которых неинформативны для нашей будущей модели (`DateCrawled`, `RegistrationMonth`, `DateCreated`,`NumberOfPictures`, `PostalCode`, `LastSeen`);


Значение признаков:

* `DateCrawled` — дата скачивания анкеты из базы
* `VehicleType` — тип автомобильного кузова
* `RegistrationYear` — год регистрации автомобиля
* `Gearbox` — тип коробки передач
* `Power` — мощность (л. с.)
* `Model` — модель автомобиля
* `Kilometer` — пробег (км)
* `RegistrationMonth` — месяц регистрации автомобиля
* `FuelType` — тип топлива
* `Brand` — марка автомобиля
* `Repaired` — была машина в ремонте или нет
* `DateCreated` — дата создания анкеты
* `NumberOfPictures` — количество фотографий автомобиля
* `PostalCode` — почтовый индекс владельца анкеты (пользователя)
* `LastSeen` — дата последней активности пользователя

## Предобработка данных

### Удаление неиформативных признаков

Уберем все неиформативные признаки, которые могут запутать модель или ухудшить точность ее прогнозов.

In [51]:
# Создание списка неинформативных признаков
uninformative_features = ['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']

df = df.drop(uninformative_features, axis=1)

df.columns

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'Repaired'],
      dtype='object')

### Обработка пропусков

In [52]:
df.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
Repaired            71154
dtype: int64

В колонках `Gearbox`, `FuelType`, `Repaired` есть пробелы, которые составляют до 20% от общей выборки. Потеря таких данных критична. По этой причине в столбцах `Gearbox` и `FuelType` мы заменим пробелы на наиболее часто встречающиеся значения. В ячейке `Repaired` отметим, что машины не биты, потому что чаще всего информация о том, битая машина или нет - отрицательна. Пробелы в столбцах `VehicleType` и `Model` оставим как есть, поскольку используемые нами модели сами определяют такие пробелы в отдельную категорию, кроме простых моделей.

In [53]:
# Замена NAN на моду
df['Gearbox'] = df['Gearbox'].fillna(df['Gearbox'].mode().values[0])
df['FuelType'] = df['FuelType'].fillna(df['FuelType'].mode().values[0])

# Замена NAN на 'no'
df['Repaired'] = df['Repaired'].fillna('no')

In [54]:
df.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox                 0
Power                   0
Model               19705
Kilometer               0
FuelType                0
Brand                   0
Repaired                0
dtype: int64

Замена NAN значений прошла корректно 

### Обработка дублей

In [55]:
df.duplicated().sum()

56547

In [56]:
df = df.drop_duplicates()

In [57]:
df.duplicated().sum()

0

Дубликаты успешно удалены

### Обработка аномальных значений 

Через цикл выведем уникальные значения в стобцах

In [58]:
for column in df.columns:
    anomaly = df[column].sort_values().unique()
    display(column)
    display(anomaly)
    display()

'Price'

array([    0,     1,     2, ..., 19998, 19999, 20000])

'VehicleType'

array(['bus', 'convertible', 'coupe', 'other', 'sedan', 'small', 'suv',
       'wagon', nan], dtype=object)

'RegistrationYear'

...

array(['cng', 'electric', 'gasoline', 'hybrid', 'lpg', 'other', 'petrol'],
      dtype=object)

'Brand'

array(['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen',
       'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai',
       'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda',
       'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart',
       'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant',
       'volkswagen', 'volvo'], dtype=object)

'Repaired'

array(['no', 'yes'], dtype=object)

Аномалии видны в столбацах `RegistrationYear` и `Power`. В столбце `RegistrationYear` имеются автомобили, поставленные на учет через 12 лет после крещения Руси, а так же автомобили из будущего. Массовое машиностроение началось с 1936 года и продолжается в наше время, поэтому в данных мы ограничимся этими годами. В части мощности ограничимся `50-1000 л.с.` т.к. машины меньшей и большей мощности в наше время редкость.

In [59]:
# Годы регистрации автомобиля от 1936 до 2022
df = df.loc[(df['RegistrationYear'] >= 1936) & (df['RegistrationYear'] <= 2024)]
# Мощность от 50 до 1000 л.с.
df = df.loc[(df['Power'] >= 50) & (df['Power'] <= 1000)]
# Цена от 100 euro
df = df.loc[df['Price'] >= 100]

display(df['RegistrationYear'].sort_values().unique())
display()
display(df['Power'].sort_values().unique())
display()
display(df.shape)

array([1937, 1941, 1942, 1943, 1945, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019])

array([  50,   51,   52,   53,   54,   55,   56,   57,   58,   59,   60,
         61,   62,   63,   64,   65,   66,   67,   68,   69,   70,   71,
         72,   73,   74,   75,   76,   77,   78,   79,   80,   81,   82,
         83,   84,   85,   86,   87,   88,   89,   90,   91,   92,   93,
         94,   95,   96,   97,   98,   99,  100,  101,  102,  103,  104,
        105,  106,  107,  108,  109,  110,  111,  112,  113,  114,  115,
        116,  117,  118,  119,  120,  121,  122,  123,  124,  125,  126,
        127,  128,  129,  130,  131,  132,  133,  134,  135,  136,  137,
        138,  139,  140,  141,  142,  143,  144,  145,  146,  147,  148,
        149,  150,  151,  152,  153,  154,  155,  156,  157,  158,  159,
        160,  161,  162,  163,  164,  165,  166,  167,  168,  169,  170,
        171,  172,  173,  174,  175,  176,  177,  178,  179,  180,  181,
        182,  183,  184,  185,  186,  187,  188,  189,  190,  191,  192,
        193,  194,  195,  196,  197,  198,  199,  2

(252128, 10)

Все аномалии удалены.

## Кодирование, масштабирование, разбивка данных

### Для регрессий 

**Разбивка данных на целевой признак и фичи**

In [60]:
features = df.drop('Price', axis=1)
target = df['Price']

**Кодирование категориальных фич**

In [61]:
features = pd.get_dummies(features, drop_first=True)

Разбивка на train и valid + test выборки.

In [62]:
(features_train, features_valid_test,
 target_train, target_valid_test) = train_test_split(features,
                                                     target,
                                                     test_size=0.4,
                                                     random_state=12345)

display(features_train.shape)
display(target_train.shape)
display(features_valid_test.shape)
display(target_valid_test.shape)

(151276, 305)

(151276,)

(100852, 305)

(100852,)

Valid + test выборку разобьем на valid и test по отдельности.

In [63]:
(features_valid, features_test,
 target_valid, target_test) = train_test_split(features_valid_test,
                                               target_valid_test,
                                               test_size=0.5,
                                               random_state=12345)

display(features_valid.shape)
display(target_valid.shape)
display(features_test.shape)
display(target_test.shape)

(50426, 305)

(50426,)

(50426, 305)

(50426,)

**Масштабирование данных**

In [64]:
# Масштабирование признаков
scale_list = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features_train[scale_list])

features_train[scale_list] = scaler.transform(features_train[scale_list])
features_valid[scale_list] = scaler.transform(features_valid[scale_list])
features_test[scale_list] = scaler.transform(features_test[scale_list])

display(features_train)
display(features_valid)
display(features_test)

/tmp/ipykernel_31/3350154884.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[scale_list] = scaler.transform(features_train[scale_list])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_31/3350154884.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_yes
185096,0.057740,-0.885473,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273854,-0.233057,-0.885473,0.609869,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
127516,-0.960048,-0.410828,-2.598058,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2401,-0.233057,0.848807,0.609869,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14691,1.947917,-0.611639,0.609869,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108171,-0.378455,-0.465594,-1.261422,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
173942,-0.378455,0.483696,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
169812,0.639333,0.136840,0.609869,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
97296,1.511722,0.027306,-2.598058,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_yes
149540,0.203138,0.721018,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
300344,0.784731,-0.885473,-0.726767,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
220186,-0.814650,0.483696,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51766,-1.105446,0.228118,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28933,0.639333,-0.136994,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8299,0.784731,0.848807,-0.726767,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
51979,0.784731,-1.159306,-2.330731,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
191684,-0.669251,-0.757684,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
256138,-0.087658,-0.885473,-0.994095,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_yes
171724,0.203138,0.574974,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
174442,0.057740,1.834609,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231988,-0.087658,-0.136994,-0.058449,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
268154,-0.233057,-0.410828,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32805,-2.123234,0.100329,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265034,0.784731,-0.995006,-2.865385,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
167622,0.203138,0.301140,0.609869,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
330466,-0.669251,-0.611639,0.609869,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
246902,-0.378455,-1.195818,0.609869,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Для деревьев

**Разбика данных на фичи и целевой признак для деревьев**

In [65]:
df_tree = df.copy()

In [66]:
df_tree['VehicleType'] = df_tree['VehicleType'].fillna(df_tree['VehicleType']
                                               .mode()
                                               .values[0])
df_tree['Model'] = df_tree['Model'].fillna(df_tree['Model']
                                   .mode()
                                   .values[0])

In [67]:
# Разбивка данных на фичи и таргет
features_tree = df_tree.drop('Price', axis=1)
target_tree = df_tree['Price']

**Кодирование категориальных фич для деревьев**

In [69]:
(features_train_tree, features_valid_tree,
 target_train_tree, target_valid_tree) = train_test_split(features_tree,
                                                          target_tree,
                                                          test_size=0.25,
                                                          random_state=12345)

display(features_train_tree.shape)
display(target_train_tree.shape)
display(features_valid_tree.shape)
display(target_valid_tree.shape)

(189096, 9)

(189096,)

(63032, 9)

(63032,)

In [70]:
(features_valid_tree, features_test_tree,
 target_valid_tree, target_test_tree) = train_test_split(features_valid_tree,
                                                         target_valid_tree,
                                                         test_size=0.5,
                                                         random_state=12345)

display(features_valid_tree.shape)
display(target_valid_tree.shape)
display(features_test_tree.shape)
display(target_test_tree.shape)

(31516, 9)

(31516,)

(31516, 9)

(31516,)

In [71]:
encoder_list = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'Repaired']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_encoder.fit(features_train_tree[encoder_list])
features_train_tree[encoder_list] = ordinal_encoder.transform(features_train_tree[encoder_list])
features_valid_tree[encoder_list] = ordinal_encoder.transform(features_valid_tree[encoder_list])
features_test_tree[encoder_list] = ordinal_encoder.transform(features_test_tree[encoder_list])
display(features_train_tree)
display(features_valid_tree)
display(features_test_tree)

/tmp/ipykernel_31/2180118040.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train_tree[encoder_list] = ordinal_encoder.transform(features_train_tree[encoder_list])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
153529,4.0,2012,0.0,122,116.0,60000,6.0,38.0,0.0
228624,0.0,2003,1.0,101,236.0,150000,2.0,24.0,0.0
86018,4.0,2008,0.0,235,15.0,150000,2.0,2.0,0.0
94469,5.0,2003,1.0,80,83.0,150000,6.0,24.0,0.0
32683,7.0,2004,1.0,155,203.0,125000,6.0,24.0,1.0
...,...,...,...,...,...,...,...,...,...
108171,4.0,2001,1.0,98,10.0,80000,6.0,19.0,0.0
173942,7.0,2001,1.0,150,170.0,150000,6.0,38.0,0.0
169812,0.0,2008,1.0,131,222.0,150000,2.0,38.0,0.0
97296,0.0,2014,1.0,125,60.0,30000,6.0,10.0,0.0


,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
71941,7.0,2009,1.0,160,122.0,100000,2.0,24.0,0.0
120901,5.0,1999,1.0,54,106.0,40000,6.0,32.0,0.0
208606,7.0,2008,1.0,156,59.0,100000,6.0,20.0,0.0
194437,4.0,1995,0.0,116,116.0,150000,6.0,38.0,0.0
302632,7.0,2002,0.0,163,29.0,150000,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
222066,4.0,2003,1.0,69,83.0,150000,2.0,24.0,1.0
245720,4.0,2016,1.0,75,116.0,150000,6.0,38.0,0.0
191684,4.0,1999,1.0,82,33.0,150000,6.0,20.0,0.0
132617,0.0,2013,1.0,105,116.0,100000,2.0,38.0,1.0


,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
51202,3.0,1991,1.0,170,166.0,150000,2.0,23.0,1.0
31521,1.0,2008,1.0,140,42.0,125000,6.0,24.0,0.0
302229,4.0,2009,1.0,102,163.0,125000,6.0,31.0,0.0
139213,4.0,2006,0.0,163,11.0,100000,2.0,2.0,0.0
108316,2.0,2003,0.0,163,76.0,150000,6.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...
136633,5.0,2014,1.0,136,80.0,20000,6.0,21.0,0.0
181956,3.0,2002,1.0,88,211.0,100000,2.0,20.0,0.0
207456,5.0,1995,1.0,50,102.0,100000,6.0,10.0,1.0
202110,1.0,2011,1.0,140,97.0,100000,2.0,38.0,0.0


### Вывод


Мы провели обработку данных для того, чтобы использовать их при обучении моделей. Нам удалось:
* удалить ненужные признаки;
* обработать пропущенные, повторяющиеся и аномальные значения;
* кодировать,  масштабировать признаки и разделить их на обучающую и валидационную выборки для моделей градиентного бустинга;
* кодировать, масштабировать признаки и их разделить на обучающую и валидационную выборки для деревьев.

## Анализ Моделей

Обучим модели на обработанных данных и оценим их качество через метрику RMSE

### DesicionTreeRegressor

**Вариант проверки №1**

In [72]:
model_tree = DecisionTreeRegressor(max_depth=10, random_state=12345)

In [73]:
%%time

model_tree.fit(features_train_tree, target_train_tree)

CPU times: user 332 ms, sys: 100 µs, total: 332 ms
Wall time: 332 ms


DecisionTreeRegressor(max_depth=10, random_state=12345)

In [74]:
%%time

predictions_valid = model_tree.predict(features_valid_tree)
rmse = mean_squared_error(target_valid_tree, predictions_valid) ** .5

display(rmse)

2056.7804397194177

CPU times: user 8.81 ms, sys: 10 µs, total: 8.82 ms
Wall time: 7.9 ms


**Вариант проверки №2**

In [75]:
model_tree_2 = DecisionTreeRegressor(max_depth=50, random_state=12345)

In [76]:
%%time

model_tree_2.fit(features_train_tree, target_train_tree)

CPU times: user 560 ms, sys: 168 µs, total: 561 ms
Wall time: 560 ms


DecisionTreeRegressor(max_depth=50, random_state=12345)

In [77]:
%%time

predictions_valid = model_tree_2.predict(features_valid_tree)
rmse = mean_squared_error(target_valid_tree, predictions_valid) ** .5

display(rmse)

2067.1659875124537

CPU times: user 16.5 ms, sys: 17 µs, total: 16.6 ms
Wall time: 15.5 ms


### LGBMRegressor

**Вариант проверки №1**

In [78]:
model_LGBM = LGBMRegressor(learning_rate=1,
                           max_depth=5,
                           metric='rmse',
                           random_state=12345)

In [79]:
%%time

model_LGBM.fit(features_train, target_train)

CPU times: user 3.52 s, sys: 150 ms, total: 3.67 s
Wall time: 3.64 s


LGBMRegressor(learning_rate=1, max_depth=5, metric='rmse', random_state=12345)

In [80]:
%%time

predictions_valid = model_LGBM.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5

display(rmse)

1807.2801812670498

CPU times: user 451 ms, sys: 33.8 ms, total: 484 ms
Wall time: 499 ms


**Вариант проверки №2**

In [81]:
model_LGBM_2 = LGBMRegressor(learning_rate=1,
                             max_depth=10,
                             metric='rmse',
                             random_state=12345)

In [82]:
%%time

model_LGBM_2.fit(features_train, target_train)

CPU times: user 4.25 s, sys: 167 ms, total: 4.41 s
Wall time: 4.39 s


LGBMRegressor(learning_rate=1, max_depth=10, metric='rmse', random_state=12345)

In [83]:
%%time

predictions_valid = model_LGBM_2.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5

display(rmse)

1760.6882827940124

CPU times: user 395 ms, sys: 57.2 ms, total: 452 ms
Wall time: 493 ms


## CatBoostRegressor

**Вариант проверки №1**

In [84]:
model_CB = CatBoostRegressor(learning_rate=1,
                             depth=5,
                             loss_function='RMSE',
                             random_state=12345)

In [85]:
%%time

model_CB.fit(features_train,
             target_train,
             verbose=False,
             plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 16.7 s, sys: 71.7 ms, total: 16.8 s
Wall time: 18.1 s


In [86]:
%%time

predictions_valid = model_CB.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5
display(rmse)

1715.3120717459763

CPU times: user 90.3 ms, sys: 37 µs, total: 90.3 ms
Wall time: 88.9 ms


**Вариант проверки №2**

In [87]:
model_CB_2 = CatBoostRegressor(learning_rate=1,
                               depth=10,
                               loss_function='RMSE',
                               random_state=12345)

In [88]:
%%time

model_CB_2.fit(features_train, target_train, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 38.8 s, sys: 173 ms, total: 39 s
Wall time: 40.3 s


In [89]:
%%time

predictions_valid = model_CB_2.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5
display(rmse)

1759.1749195741265

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 200 ms


### Вывод:

По итогам обучения мы имеем следующие показатели 

| Вариант реализации | Показатели | DecisionTreeRegressor | LGBMRegressor | CatBoostRegressor|  
| --- | --- | --- |--- | --- |
| Вариант №1  | RMSE | 2056.78 |1807.28 | 1715.31 |
|   | Скорость обучения | 332 ms | 3.88 s | 19.4 s |
|   | Скорость предсказания | 7.67 ms | 409 ms | 99.1 ms |
| Вариант №2  | RMSE | 2067.16 | 1760.69 | 1759.18 |
|   | Скорость обучения | 568 ms | 4.27 s | 42.2 s |
|   | Скорость предсказания | 17.6 ms | 494 ms | 264 ms |

Как видно из итоговой таблицы, `DecisionTreeRegressor` с увеличением глубины обучения переобучилось и стало плохо предсказывать. То же самое произошло и с моделью `CatBoostRegressor` - ее время обучения и предсказания увеличилось, а ключевая метрика ухудшилась. `LGBMRegressor` с увеличение глубины улучшил свои показатели, но ненамного.

**Проверим лучшую модель на тестовой выборке**

In [90]:
%%time

predictions_test = model_CB.predict(features_test)
rmse = mean_squared_error(target_test, predictions_test) ** .5
display(rmse)

1679.1692521246946

CPU times: user 81.4 ms, sys: 0 ns, total: 81.4 ms
Wall time: 86.2 ms


RMSE на тестовой выборке составляет 1679.17 Евро.

## Итоговый вывод:

Исходя из проведенного анализа данных и построения модели машинного обучения можно сделать вывод, что в соответствии с критериями, установленными заказчиком, а именно:

* качество предсказания;
* время обучения модели;
* время предсказания модели.

Самой подходящей моделью является первый вариант `CatBoostRegressor` с max_depth=5, т.к. скорость ее предсказания достаточно высока при низкой RMSE. Обучение модели проходит примерно в 5 раз дольше, чем обучение `LGBMRegressor`, однако, т.к. данные статичные, то переобучение модели можно производить нединамично и эта метрика менее важная, чем отлик модели на запрос пользователя. RMSE на тестовой выборке показывает значение 1679.17 евро, что несильно отличается от показателя на валидационной выборке.